<a href="https://colab.research.google.com/github/yashveersinghsohi/Hands_On_ML_Book_Practice/blob/master/Chapter_14/Chapter14_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNs and Pooling Layers

In [1]:
from sklearn.datasets import load_sample_images
import tensorflow as tf

In [2]:
images = load_sample_images()['images']
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale=1/255)(images)
images.shape

TensorShape([2, 70, 120, 3])

In [3]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=(7, 7))
fmaps = conv_layer(images)
fmaps.shape

TensorShape([2, 64, 114, 32])

In [4]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=(7, 7), padding='same')
fmaps = conv_layer(images)
fmaps.shape

TensorShape([2, 70, 120, 32])

In [5]:
conv_layer.weights

[<KerasVariable shape=(7, 7, 3, 32), dtype=float32, path=conv2d_1/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=conv2d_1/bias>]

In [8]:
kernels, biases = conv_layer.get_weights()
kernels.shape, biases.shape

((7, 7, 3, 32), (32,))

In [9]:
pool_layer = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
pool_op = pool_layer(fmaps)
pool_op.shape

TensorShape([2, 35, 60, 32])